In [130]:
from numpy import*

In [131]:
def loadDataSet():
 return[['M','O','N','K','E','Y'],['D','O','N','K','E','Y'],['M','A','K','E'],['M','U','C','K','Y'],['C','O','O','K','I','E']]


In [132]:
def createC1(dataSet):
 C1=[]
 for transaction in dataSet:
  for item in transaction:
    if not[item] in C1:
     C1.append([item])
 C1.sort()
 return list(map(frozenset,C1)) 

In [133]:
def scanD(D,Ck,minSupport):
   ssCnt={}
   for tid in D:
      for can in Ck:
        if can.issubset(tid):
            if not can in ssCnt:
                 ssCnt[can]=1
            else:
                 ssCnt[can]+=1
   numItems=float(len(D))
   retList=[]
   supportData={}
   for key in ssCnt:
        support=ssCnt[key]/numItems
        if support>=minSupport:
             retList.insert(0,key)
        supportData[key]=support
   return retList,supportData

In [134]:
dataSet=loadDataSet()
dataSet


[['M', 'O', 'N', 'K', 'E', 'Y'],
 ['D', 'O', 'N', 'K', 'E', 'Y'],
 ['M', 'A', 'K', 'E'],
 ['M', 'U', 'C', 'K', 'Y'],
 ['C', 'O', 'O', 'K', 'I', 'E']]

In [135]:
C1=createC1(dataSet)

In [136]:
C1

[frozenset({'A'}),
 frozenset({'C'}),
 frozenset({'D'}),
 frozenset({'E'}),
 frozenset({'I'}),
 frozenset({'K'}),
 frozenset({'M'}),
 frozenset({'N'}),
 frozenset({'O'}),
 frozenset({'U'}),
 frozenset({'Y'})]

In [137]:
D=list(map(set,dataSet))

In [138]:
D

[{'E', 'K', 'M', 'N', 'O', 'Y'},
 {'D', 'E', 'K', 'N', 'O', 'Y'},
 {'A', 'E', 'K', 'M'},
 {'C', 'K', 'M', 'U', 'Y'},
 {'C', 'E', 'I', 'K', 'O'}]

In [139]:
L1,suppDat0=scanD(D,C1,0.6)
L1

[frozenset({'Y'}),
 frozenset({'O'}),
 frozenset({'M'}),
 frozenset({'K'}),
 frozenset({'E'})]

In [140]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: 
                retList.append(Lk[i] | Lk[j]) 
    return retList


In [141]:
def apriori(dataSet, minSupport = 0.6):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData


In [142]:
L,suppData=apriori(dataSet)

In [143]:
L

[[frozenset({'Y'}),
  frozenset({'O'}),
  frozenset({'M'}),
  frozenset({'K'}),
  frozenset({'E'})],
 [frozenset({'E', 'K'}),
  frozenset({'K', 'M'}),
  frozenset({'E', 'O'}),
  frozenset({'K', 'O'}),
  frozenset({'K', 'Y'})],
 [frozenset({'E', 'K', 'O'})],
 []]

In [144]:
L[0]

[frozenset({'Y'}),
 frozenset({'O'}),
 frozenset({'M'}),
 frozenset({'K'}),
 frozenset({'E'})]

In [145]:
L[1]

[frozenset({'E', 'K'}),
 frozenset({'K', 'M'}),
 frozenset({'E', 'O'}),
 frozenset({'K', 'O'}),
 frozenset({'K', 'Y'})]

In [146]:
L[2]

[frozenset({'E', 'K', 'O'})]

In [147]:
L[3]

[]

In [148]:
aprioriGen(L[0],2)

[frozenset({'O', 'Y'}),
 frozenset({'M', 'Y'}),
 frozenset({'K', 'Y'}),
 frozenset({'E', 'Y'}),
 frozenset({'M', 'O'}),
 frozenset({'K', 'O'}),
 frozenset({'E', 'O'}),
 frozenset({'K', 'M'}),
 frozenset({'E', 'M'}),
 frozenset({'E', 'K'})]

In [149]:
def generateRules(L,supportData,minConf=0.7):
 bigRuleList=[]
 for i in range(1,len(L)):
  for freqSet in L[i]:
    H1=[frozenset([item])for item in freqSet]
    if(i>1):
     rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf)
    else:
     calcConf(freqSet,H1,supportData,bigRuleList,minConf)
 return bigRuleList


In [150]:
def calcConf(freqSet,H,supportData,brl,minConf=0.7):
 prunedH=[]
 for conseq in H:
  conf=supportData[freqSet]/supportData[freqSet-conseq]
  if conf>=minConf:
   print(freqSet-conseq,'-->',conseq,'conf:',conf)
   brl.append((freqSet-conseq,conseq,conf))
   prunedH.append(conseq)
 return prunedH


In [151]:
def rulesFromConseq(freqSet,H,supportData,brl,minConf=0.7):
 m=len(H[0])
 if(len(freqSet)>(m+1)):
  Hmp1=aprioriGen(H,m+1)
  Hmp1=calcConf(freqSet,Hmp1,supportData,brl,minConf)
 if(len(Hmp1)>1):
   rulesFromConseq(freqSet,Hmp1,supportData,brl,minConf)


In [152]:
L,suppData=apriori(dataSet,minSupport=0.6)

In [153]:
rules = generateRules(L, suppData, minConf=0.7)

for rule in rules:
    antecedent, consequent, confidence = rule
    print(f"{antecedent} --> {consequent} conf: {confidence}")


frozenset({'K'}) --> frozenset({'E'}) conf: 0.8
frozenset({'E'}) --> frozenset({'K'}) conf: 1.0
frozenset({'M'}) --> frozenset({'K'}) conf: 1.0
frozenset({'O'}) --> frozenset({'E'}) conf: 1.0
frozenset({'E'}) --> frozenset({'O'}) conf: 0.7499999999999999
frozenset({'O'}) --> frozenset({'K'}) conf: 1.0
frozenset({'Y'}) --> frozenset({'K'}) conf: 1.0
frozenset({'O'}) --> frozenset({'E', 'K'}) conf: 1.0
frozenset({'E'}) --> frozenset({'O', 'K'}) conf: 0.7499999999999999


UnboundLocalError: cannot access local variable 'Hmp1' where it is not associated with a value